In [10]:
import os
from sys import getsizeof

In [11]:
import cudf
import cupy
from tqdm import tqdm
import numpy as np
import gc
import xgboost as xgb
from utils import amex_metric_np
from pathlib import Path

cudf.__version__, xgb.__version__

('23.12.01', '1.7.6')

In [12]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)

# Please register kaggle and install kaggle API by:
- `pip install kaggle`
- complete [authentication](https://www.kaggle.com/docs/api)

In [13]:
PATH = 'raid/data/ml/kaggle/amex'
os. getcwd()

'/mnt/d/workspace/Master/proiect_dmcf/triton_amex'

In [4]:
os.umask(0)
Path(PATH).mkdir(parents=True,exist_ok=True)

In [5]:
cmd = f'kaggle datasets download -d raddar/amex-data-integer-dtypes-parquet-format -p {PATH}'

In [20]:
os.system(cmd)

100%|██████████| 4.07G/4.07G [02:31<00:00, 28.9MB/s]


0

In [14]:
os.listdir(PATH)

['amex-data-integer-dtypes-parquet-format.zip',
 'test.parquet',
 'train.parquet',
 'train_labels.csv',
 'train_labels.csv.zip']

In [40]:
cmd = f'cd {PATH} && unzip -o amex-data-integer-dtypes-parquet-format.zip'
os.system(cmd)

Archive:  amex-data-integer-dtypes-parquet-format.zip
  inflating: test.parquet            
  inflating: train.parquet           


0

In [15]:
os.listdir(PATH)

['amex-data-integer-dtypes-parquet-format.zip',
 'test.parquet',
 'train.parquet',
 'train_labels.csv',
 'train_labels.csv.zip']

# Basic EDA

In [16]:

%%time
train_cudf = cudf.read_parquet(f'{PATH}/train.parquet')
print(train_cudf.shape)
train_cudf.head()

MemoryError: ignored

In [ ]:
train = dask_cudf.from_cudf(train_cudf, npartitions=16)
print(train.shape)
train.head()

In [8]:
%%time
count_df = train.groupby('customer_ID').size().to_frame('num_profiles')
count_df.head()

CPU times: user 109 ms, sys: 0 ns, total: 109 ms
Wall time: 115 ms


,num_profiles
customer_ID,
c761f5f5b15e563daa67f0a41c3ec2a870d3c9daaadf0cd11dd808d3aaa82c46,13
e16b5594d9dce9ebd2f8e0d7074391736b2641afa9e349f67a53f7cc780c120b,13
8c846c26e1f1d4afa04977155c41bc3b6bb77c72efc5db3f592ec3d72f12cfdc,13
463e8a9b5b0161764bbbb0b5b58956bb8ebff6244219b21ac257a07364fa8dd9,13
92bbe3e2a159bcc838b86241471eb14153c8d712b6647feffbe49d5266cdfd3f,13


In [9]:
count_df.num_profiles.max()

13

In [5]:
%%time
train['S_2'] = cudf.to_datetime(train['S_2'])
train.head()

CPU times: user 267 ms, sys: 10.3 ms, total: 277 ms
Wall time: 272 ms


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,<NA>,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,<NA>,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,<NA>,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,<NA>,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,<NA>,0,0.008126,0


In [11]:
train.S_2.min(), train.S_2.max()

(numpy.datetime64('2017-03-01T00:00:00.000000000'),
 numpy.datetime64('2018-03-31T00:00:00.000000000'))

## Download the training data labels

In [30]:
cmd = f'kaggle competitions download -c amex-default-prediction -f train_labels.csv -p {PATH}/'
os.system(cmd)

 62%|██████▏   | 10.0M/16.2M [00:00<00:00, 19.9MB/s]

100%|██████████| 16.2M/16.2M [00:00<00:00, 17.7MB/s]


0

In [31]:
cmd = f'cd {PATH} && unzip train_labels.csv.zip'
os.system(cmd)

Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


0

In [6]:
%%time
trainL_cudf = cudf.read_csv(f'{PATH}/train_labels.csv')
print(trainL_cudf.shape)
trainL_cudf.head()

(458913, 2)
CPU times: user 296 ms, sys: 421 ms, total: 717 ms
Wall time: 2.24 s


,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0


In [ ]:
trainl = dask_cudf.from_cudf(trainL_cudf, npartitions=16)
print(trainl.shape)
trainl.head()

In [13]:
trainl['target'].value_counts()

0    340085
1    118828
Name: target, dtype: int64

In [14]:
print(getsizeof(train))
print(getsizeof(trainl))

48
48


In [ ]:
%%time
train = train.merge(trainl, on='customer_ID', how='left')
print(train.shape)
train.head()

In [ ]:
train.persist()
train.head()

In [ ]:
train['cid'], _ = train.customer_ID.factorize()

In [ ]:
mask = train['cid']%4 == 0
tr,va = train.loc[~mask],train.loc[mask]
print("Verify target distribution is consistent across tr and va")
print(tr['target'].mean(), va['target'].mean())

Verify target distribution is consistent across tr and va
0.2493462806763533 0.24835027876095958
